In [ ]:
# ==============================================================================
# Tutorial 2: Hierarchical Cell Phenotyping
# ==============================================================================
# Goal: Classify single-cell patches using the two-stage PhenoSSP framework.
# Logic:
#   Stage 1: Coarse Classification (Epithelial vs. Immune vs. Other)
#   Stage 2: Fine-grained Classification (Only for Immune cells)
# ==============================================================================

import os
import sys
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# --- 1. Setup Environment ---
# Add project root to path to import 'phenossp' modules
current_dir = os.path.dirname(os.path.abspath("__file__")) # For .py script
# current_dir = os.getcwd() # Use this for Notebook
project_root = os.path.dirname(current_dir)
sys.path.insert(0, project_root)

from phenossp.vision_transformer import vit_small

# Configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
WEIGHTS_PATH = os.path.join(project_root, 'phenossp', 'weights', 'phenossp_final.pth')
DEMO_DATA_DIR = os.path.join(project_root, 'demo_data', 'patches')

# Define Label Maps (Must match training)
COARSE_LABELS = {0: 'Epithelial', 1: 'Immune', 2: 'Other'}
IMMUNE_LABELS = {
    0: 'CD3+CD4+CD8-',   # Helper T
    1: 'CD3+CD4-CD8+',   # Cytotoxic T
    2: 'CD3-CD4+CD8-',   # Other CD4+
    3: 'CD3+CD4-CD8-',   # Other T
    4: 'CD4+FoxP3+'      # Treg
}

print(f"🔧 Device: {DEVICE}")
print(f"📂 Loading weights from: {WEIGHTS_PATH}")

In [ ]:
# --- 2. Define the Hierarchical Model Wrapper ---
# In a real scenario, this might be imported from `phenossp.model`
# Here we define it explicitly to show the logic to the reviewer.

class PhenoSSP_Hierarchical(nn.Module):
    def __init__(self, num_coarse=3, num_immune=5, embed_dim=384):
        super().__init__()
        # 1. Backbone (ViT-S/16)
        self.backbone = vit_small(patch_size=16, embed_dim=embed_dim, num_classes=0) 
        
        # 2. Stage 1: Coarse Head
        self.coarse_head = nn.Sequential(
            nn.Linear(embed_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_coarse)
        )
        
        # 3. Stage 2: Immune Expert Head
        self.immune_head = nn.Sequential(
            nn.Linear(embed_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_immune)
        )

    def forward(self, x):
        # Extract features (Batch, 384)
        features = self.backbone(x)
        
        # Always run coarse classification
        coarse_logits = self.coarse_head(features)
        
        # Run immune classification (we compute it for all, but only use it if Coarse==Immune)
        immune_logits = self.immune_head(features)
        
        return coarse_logits, immune_logits

# Initialize and Load Weights
def load_model(weights_path):
    model = PhenoSSP_Hierarchical().to(DEVICE)
    
    if os.path.exists(weights_path):
        # Allow loading standard state_dict or checkpoint dict
        checkpoint = torch.load(weights_path, map_location=DEVICE)
        if 'model_state_dict' in checkpoint:
            state_dict = checkpoint['model_state_dict']
        else:
            state_dict = checkpoint
            
        # Handle prefix matching if necessary (e.g. removing 'module.')
        clean_state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
        
        # Load (strict=False allows flexibility if auxiliary keys exist)
        model.load_state_dict(clean_state_dict, strict=False)
        print("✅ Model weights loaded successfully.")
    else:
        print(f"⚠️ Warning: Weight file not found at {weights_path}. Initializing with random weights for demo.")
        
    model.eval()
    return model

model = load_model(WEIGHTS_PATH)

In [ ]:
# --- 3. Data Loading Helper ---
def load_patch(patch_path):
    """
    Loads a .npy patch and prepares it for the model.
    Input NPY shape: (7, 64, 64) -> Output Tensor: (1, 7, 64, 64)
    """
    try:
        patch = np.load(patch_path)
        
        # Normalize (Simple Min-Max for demo)
        # In production, use the exact normalization statistics from training
        patch = (patch - patch.min()) / (patch.max() - patch.min() + 1e-8)
        
        # Convert to Tensor
        tensor = torch.from_numpy(patch).float().unsqueeze(0) # Add batch dim
        return tensor.to(DEVICE), patch
    except Exception as e:
        print(f"Error loading {patch_path}: {e}")
        return None, None

In [ ]:
# --- 4. Run Inference Pipeline ---
print("\n🚀 Starting Inference on Demo Patches...\n")

# Get list of demo patches
patch_files = [f for f in os.listdir(DEMO_DATA_DIR) if f.endswith('.npy')]
patch_files.sort()

# Visualize predictions
fig, axes = plt.subplots(1, min(5, len(patch_files)), figsize=(15, 4))
if len(patch_files) == 1: axes = [axes]

for i, filename in enumerate(patch_files[:5]):
    # A. Load Data
    patch_path = os.path.join(DEMO_DATA_DIR, filename)
    input_tensor, raw_patch = load_patch(patch_path)
    
    if input_tensor is None: continue
    
    # B. Model Prediction
    with torch.no_grad():
        coarse_logits, immune_logits = model(input_tensor)
        
        # -- Stage 1 Decision --
        coarse_probs = torch.softmax(coarse_logits, dim=1)
        coarse_pred_idx = torch.argmax(coarse_probs, dim=1).item()
        coarse_label = COARSE_LABELS[coarse_pred_idx]
        coarse_conf = coarse_probs[0, coarse_pred_idx].item()
        
        # -- Stage 2 Decision (Conditional) --
        final_label = coarse_label
        final_conf = coarse_conf
        is_immune = (coarse_label == 'Immune')
        
        if is_immune:
            immune_probs = torch.softmax(immune_logits, dim=1)
            immune_pred_idx = torch.argmax(immune_probs, dim=1).item()
            fine_label = IMMUNE_LABELS[immune_pred_idx]
            fine_conf = immune_probs[0, immune_pred_idx].item()
            
            final_label = f"{fine_label} (Immune)"
            final_conf = fine_conf  # Or average of both, depending on logic

    # C. Print Result
    print(f"[{filename}] -> Stage 1: {coarse_label} ({coarse_conf:.2f})", end="")
    if is_immune:
        print(f" -> Stage 2: {fine_label} ({fine_conf:.2f})")
    else:
        print(" -> Stage 2: Skipped (Non-Immune)")

    # D. Visualize
    # Composite: R=PanCK, G=CD8, B=DAPI
    rgb = np.zeros((64, 64, 3))
    rgb[..., 0] = raw_patch[3] # PanCK
    rgb[..., 1] = raw_patch[1] # CD8
    rgb[..., 2] = raw_patch[0] # DAPI
    
    if i < len(axes):
        ax = axes[i]
        ax.imshow(np.clip(rgb * 1.5, 0, 1))
        ax.set_title(f"Pred: {final_label.split(' ')[0]}\nConf: {final_conf:.2f}", fontsize=10, color='green')
        ax.axis('off')

plt.tight_layout()
plt.show()